# Semantic Parsing Final Project
Link to the paper: https://aclanthology.org/P16-1004.pdf

Read through the paper fully before starting the assignment!

In [1]:
import torch
import torch.nn as nn

from google.colab import drive
drive.mount('/content/drive')

FILEPATH = "drive/MyDrive/data/"

Mounted at /content/drive


# Data Downloading
This cell obtains the pre-processed Jobs dataset (see the paper) that you will be using to train and evaluate your model. (Pre-processed meaning that argument identification, section 3.6, has already been done for you). You should only need to run this cell ***once***. Feel free to delete it after running. Create a folder in your Google Drive in which the code below will store the pre-processed data needed for this project. Modify `FILEPATH` above to direct to said folder. It should start with `drive/MyDrive/...`, feel free to take a look at previous assignments that use mounting Google Drive if you can't remember what it should look like. *Make sure the data path ends with a slash character ('/').* The below code will access the zip file containing the pre-processed Jobs dataset from the paper and extract the files into your folder! Feel free to take a look at the `train.txt` and `test.txt` files to see what the data looks like. :)

In [2]:
import requests
import io
import zipfile

# https://stackoverflow.com/questions/31126596/saving-response-from-requests-to-file
response = requests.get('http://dong.li/lang2logic/seq2seq_jobqueries.zip')
if response.status_code == 200:
  # https://stackoverflow.com/questions/3451111/unzipping-files-in-python
  with zipfile.ZipFile(io.BytesIO(response.content), "r") as zip_ref:
    zip_ref.extractall(FILEPATH)
  print("Extraction completed.")
else:
  print("Failed to download the zip file.")

Extraction completed.


# Data Pre-processing
The following code is defined for you! It extracts the queries (inputs to your Seq2Seq model) and logical forms (expected outputs) from the training and testing files. It also does important pre-processing such as padding the queries and logical forms and turns the words into vocab indices. **Look over and understand this code before you start the assignment!**

In [3]:
def extract_file(filename):
  """
  Extracts queries and corresponding logical forms from either
  train.txt or test.txt. (Feel free to take a look at the files themselves
  in your Drive!)

  Parameters
  ----------
  filename : str
      name of the file to extract from

  Returns
  ----------
  tuple[list[list[str]], list[list[str]]]
      a tuple of a list of queries and their corresponding logical forms
      each in the form of a list of string tokens
  """
  queries, logical_forms = [], []
  with open(FILEPATH + filename) as f:
    for line in f:
      line = line.strip() # remove new line character
      query, logical_form = line.split('\t')

      query = query.split(' ')[::-1] # reversed inputs are used the paper (section 4.2)
      logical_form = ["<s>"] + logical_form.split(' ') + ["</s>"]

      queries.append(query)
      logical_forms.append(logical_form)
  return queries, logical_forms

query_train, lf_train = extract_file('train.txt') # 500 instances
query_test, lf_test = extract_file('test.txt') # 140 instances

In [4]:
from collections import Counter

query_vocab = Counter()
for l in query_train:
  query_vocab.update(l)

query_word2idx = {}
for w, c in query_vocab.items():
  if c >= 2:
    query_word2idx[w] = len(query_word2idx)
query_word2idx['<UNK>'] = len(query_word2idx)
query_word2idx['<PAD>'] = len(query_word2idx)
query_idx2word = {i:word for word,i in query_word2idx.items()}

query_vocab = list(query_word2idx.keys())

lf_vocab = Counter()
for lf in lf_train:
  lf_vocab.update(lf)

lf_vocab['<UNK>'] = 0
lf_vocab['<PAD>'] = 0
lf_idx2word = {i:word for i, word in enumerate(lf_vocab.keys())}
lf_word2idx = {word:i for i, word in lf_idx2word.items()}

# print(list(query_word2idx.items())[:10]) # Now Quey stands for  input
# print(list(query_idx2word.items())[:10])

# print(list(lf_idx2word.items())[:10]) # Now LF stands for  output
# print(list(lf_idx2word.items())[:10])


In [5]:
query_train_tokens = [[query_word2idx.get(w, query_word2idx['<UNK>']) for w in l] for l in query_train]
query_test_tokens = [[query_word2idx.get(w, query_word2idx['<UNK>']) for w in l] for l in query_test]

lf_train_tokens = [[lf_word2idx.get(w, lf_word2idx['<UNK>']) for w in l] for l in lf_train]
lf_test_tokens = [[lf_word2idx.get(w, lf_word2idx['<UNK>']) for w in l] for l in lf_test]

def pad(seq, max_len, pad_token_idx):
  """
  Pads a given sequence to the max length using the given padding token index

  Parameters
  ----------
  seq : list[int]
      sequence in the form of a list of vocab indices
  max_len : int
      length sequence should be padded to
  pad_token_idx
      vocabulary index of the padding token

  Returns
  ----------
  list[int]
      padded sequence
  """
  seq = seq[:max_len]
  padded_seq = seq + (max_len - len(seq)) * [pad_token_idx]
  return padded_seq

query_max_target_len = max([len(i) for i in query_train_tokens])
query_train_tokens = [pad(i, query_max_target_len, query_word2idx['<PAD>']) for i in query_train_tokens]
query_test_tokens = [pad(i, query_max_target_len, query_word2idx['<PAD>']) for i in query_test_tokens]

lf_max_target_len = int(max([len(i) for i in lf_train_tokens]) * 1.5)
lf_train_tokens = [pad(i, lf_max_target_len, lf_word2idx['<PAD>']) for i in lf_train_tokens]
lf_test_tokens = [pad(i, lf_max_target_len, lf_word2idx['<PAD>']) for i in lf_test_tokens]

# Data Loading
The following code creates a JobsDataset and DataLoaders to use with your implemented model. Take a look at the main function at the end of this stencil to see how they are used in context.

In [6]:
from torch.utils.data import Dataset, DataLoader, default_collate

class JobsDataset(Dataset):
  """Defines a Dataset object for the Jobs dataset to be used with Dataloader"""
  def __init__(self, queries, logical_forms):
    """
    Initializes a JobsDataset

    Parameters
    ----------
    queries : list[list[int]]
        a list of queries, which have been tokenized and padded, in the form
        of a list of vocab indices
    logical_forms : list[list[int]]
        a list of corresponding logical forms, which have been tokenized and
        padded, in the form of a list of vocab indices
    """
    self.queries = queries
    self.logical_forms = logical_forms

  def __len__(self) -> int:
    """
    Returns the amount of paired queries and logical forms in the dataset

    Returns
    ----------
    int
        length of the dataset
    """
    return len(self.queries)

  def __getitem__(self, idx: int) -> tuple[list[int], list[int]]:
    """
    Returns a paired query and logical form at the specified index

    Parameters
    ----------
    idx : int
        specified index of the dataset

    Returns
    ----------
    tuple[list[int], list[int]]
        paired query and logical form at the specified index, in the form of
        a list of vocab indices
    """
    return self.queries[idx], self.logical_forms[idx]

def build_datasets() -> tuple[JobsDataset, JobsDataset]:
  """
  Builds a train and a test dataset from the queries and logical forms
  train and test tokens

  Returns
  ----------
  tuple[JobsDataset, JobsDataset]
      a training and testing JobsDataset
  """
  jobs_train = JobsDataset(queries=query_train_tokens, logical_forms=lf_train_tokens)
  jobs_test = JobsDataset(queries=query_test_tokens, logical_forms=lf_test_tokens)
  return jobs_train, jobs_test

def collate(batch : list[tuple[list[int], list[int]]]) -> tuple[torch.Tensor, torch.Tensor]:
  """
  Used as collate_fn when creating the Dataloaders from the dataset

  Parameters
  ----------
  batch : list[tuple[list[int], list[int]]]
      a list of outputs of __getitem__

  Returns
  ----------
  tuple[torch.Tensor, torch.Tensor]
      a batched set of input sequences and a batched set of target sequences
  """
  src, tgt = default_collate(batch)
  return torch.stack(src), torch.stack(tgt)

def build_dataloaders(dataset_train: JobsDataset, dataset_test: JobsDataset,
                      train_batch_size: int) -> tuple[DataLoader, DataLoader]:
  """
  Used as collate_fn when creating the Dataloaders from the dataset, batching
  the training data according to the inputted batch size and batching the
  testing data with a batch size of 1

  Parameters
  ----------
  dataset_train : JobsDataset
      training dataset
  dataset_test : JobsDataset
      testing dataset
  train_batch_size : int
      batch size to be used during training

  Returns
  ----------
  tuple[DataLoader, DataLoader]
      a training and testing DataLoader
  """
  dataloader_train = DataLoader(dataset_train, batch_size=train_batch_size, shuffle=True, collate_fn=collate)
  dataloader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=collate)
  return dataloader_train, dataloader_test

# See the dataloader Now!

In [7]:
# 1. 加载数据集
jobs_train, jobs_test = build_datasets()

# 2. 构建 DataLoader
train_dataloader, test_dataloader = build_dataloaders(jobs_train, jobs_test, train_batch_size=20)

# 3. 打印 DataLoader 的内容
print("Printing a few samples from the train DataLoader:")
for i, (queries, logical_forms) in enumerate(train_dataloader):
    print(f"Batch {i+1}:")
    print(f"Queries Shape (Input): {queries.shape}")
    print(f"Logical Forms Shape (Target): {logical_forms.shape}")
    break  # 只打印第一个 batch

# 遍历 DataLoader 的前几个 batch
for i, (queries, logical_forms) in enumerate(train_dataloader):
    print(f"Batch {i+1}:")
    print(f"Queries: {queries}")
    print(f"Logical Forms: {logical_forms}")

    break

for i, (queries, logical_forms) in enumerate(train_dataloader):
    print(f"Batch {i+1}:")
    print(f"Queries (Input): {queries}")
    print(f"Logical Forms (Target): {logical_forms}")
    break  # 打印一个 batch 即可



Printing a few samples from the train DataLoader:
Batch 1:
Queries Shape (Input): torch.Size([20, 20])
Logical Forms Shape (Target): torch.Size([64, 20])
Batch 1:
Queries: tensor([[ 45,  13,  99,  13,  43,  13,  27,  37,  84,  20, 102,  13,  25,  21,
          21,  39,  84,  75,   7,  67],
        [ 21,  33,  50,  33,  59,  14,  41,  34,  89,   2,  11,  33,   4,  16,
          16,  59,  89,  33,  59,  18],
        [ 33,  14,  92,  14,  25,   9,  42,  24,  18,   9,  12,  14,  84,   2,
           2,   7,  18,  14, 116,  59],
        [ 27,  15,  76,  97,  14,  24,  40,  38, 100,   3,  21,  50,  85,  20,
          20,   8, 100,  50,  63,   9],
        [  2,  16,  63,  35,  35,   2,   9,  39, 101,   5, 103,  51,  59,   9,
           9,   9, 101,  51,  21,  17],
        [ 20,   9,   7,   7,   9,  20,   3,  40,  67,   6,  51,  35,  13,   3,
           3,  24,  67,  92,  12,   5],
        [  9,   3,  14,   8,  17, 125, 125, 125,  33, 125,   2,  11,  33, 125,
         125,   2,  33,   7,  52,  

# Import: Import all packages that I might used here!





In [8]:
import torch.nn as nn
import random
from torch import optim
from timeit import default_timer as timer
from torch.optim import RMSprop
from torch.optim.lr_scheduler import StepLR

# TODO: Define your model here!

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from timeit import default_timer as timer

QUERY_VOCAB_LEN = len(query_vocab)  # Input vocabulary size
LF_VOCAB_LEN = len(lf_vocab)  # Output vocabulary size
emb_dim = 128  # Embedding dimension
hid_dim = 256  # Hidden state dimension for LSTM
n_layers = 2  # Number of LSTM layers
device = "cuda" if torch.cuda.is_available() else "cpu"

class Attention(nn.Module):
    def __init__(self, hid_dim: int = 256, output_dim: int = LF_VOCAB_LEN):
        super().__init__()
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=0)
        self.w0 = nn.Linear(hid_dim, output_dim)  # Final output
        self.w1 = nn.Linear(hid_dim, hid_dim)
        self.w2 = nn.Linear(hid_dim, hid_dim)

    def forward(self, hidden: torch.FloatTensor, enc_outputs: torch.FloatTensor, dec_outputs: torch.FloatTensor):
        """
        Attention mechanism to compute context vector.
        """
        return self.w0(self.tanh(
            self.w1(dec_outputs)
             + self.w2(
                 torch.sum(self.softmax(enc_outputs * hidden[-1])
                 * enc_outputs, dim=0) )))


class Seq2Seq(nn.Module):
    """
    Seq2Seq model combining Encoder, Decoder, and Attention modules.
    """

    class Encoder(nn.Module):
        def __init__(self, input_dim: int, emb_dim: int, hid_dim: int, n_layers: int):
            super().__init__()
            # Define a two layer lstm, fetch hidden using hidden[0], hidden[1], cell[0],cell[1]
            self.lstm = nn.LSTM(input_size=emb_dim, hidden_size=hid_dim, num_layers=n_layers)
            # According to rubric, use nn.Embedding
            self.embedding = nn.Embedding(num_embeddings=input_dim, embedding_dim=emb_dim)

        def forward(self, enc_input: torch.Tensor) -> dict:
            embedding = self.embedding(enc_input)
            # Use a state_dict to store returned info and manage data
            outputs, (hidden, cell) = self.lstm(embedding)
            return {"outputs": outputs, "hidden": hidden, "cell": cell}

    class Decoder(nn.Module):
        def __init__(self, output_dim: int, emb_dim: int, hid_dim: int, n_layers: int):
            super().__init__()
            # Define a two layer lstm, fetch hidden using hidden[0], hidden[1], cell[0],cell[1]
            self.lstm = nn.LSTM(input_size=emb_dim, hidden_size=hid_dim, num_layers=n_layers)
            # According to rubric, use nn.Embedding
            self.embedding = nn.Embedding(num_embeddings=output_dim, embedding_dim=emb_dim)

        def forward(self, dec_input: torch.Tensor, state_dict: dict) -> dict:
            embedding = self.embedding(dec_input)
            # Read state_dict passed from encoder, in this case, we use hidden output to initialize decoder!
            outputs, (hidden, cell) = self.lstm(embedding, (state_dict["hidden"], state_dict["cell"]))
            state_dict["outputs"] = outputs
            state_dict["hidden"] = hidden
            state_dict["cell"] = cell
            return state_dict

    def __init__(self, device: str, input_dim: int, output_dim: int, emb_dim: int, hid_dim: int, n_layers: int):
        super().__init__()
        self.encoder = self.Encoder(input_dim, emb_dim, hid_dim, n_layers).to(device)
        self.decoder = self.Decoder(output_dim, emb_dim, hid_dim, n_layers).to(device)
        self.attention = Attention(hid_dim=hid_dim, output_dim=output_dim).to(device)
        self.device = device

        # Loss function and optimizer
        self.criterion = nn.NLLLoss(ignore_index=lf_word2idx["<PAD>"])
        self.optimizer = optim.RMSprop(self.parameters(), lr=0.001)

    def get_loss(self):
        return self.criterion

    def get_optimizer(self):
        return self.optimizer

    def forward(self, input_batch: torch.Tensor, query_batch: torch.Tensor, teacher_forcing_ratio: float = 1.0) -> torch.Tensor:
        query_size, batch_size = query_batch.shape
        query_vocab_size = self.decoder.embedding.num_embeddings
        outputs = torch.zeros(query_size, batch_size, query_vocab_size).to(self.device)

        # Encoder forward pass
        state_dict_enc = self.encoder(input_batch)
        enc_outputs, hidden, cell = state_dict_enc["outputs"], state_dict_enc["hidden"], state_dict_enc["cell"]

        # Start decoding with <SOS> token
        query = query_batch[0].unsqueeze(0)

        for i in range(1, query_size):
            state_dict_dec = self.decoder(query, state_dict_enc) # in this case, we use hidden output for encoder to initialize decoder!
            outputs[i] = self.attention(state_dict_dec["hidden"], enc_outputs, state_dict_dec["outputs"])
            best_pred = torch.argmax(outputs[i], dim=-1)

            if teacher_forcing_ratio == 1.0: # Training and Use ground truth
                query = query_batch[i].unsqueeze(0)
            else: # Testing and  Use model's prediction
                query = best_pred.unsqueeze(0)

        return outputs


# Model creation function
def create_model():
    return Seq2Seq(
        device=device,
        input_dim=QUERY_VOCAB_LEN,
        output_dim=LF_VOCAB_LEN,
        emb_dim=emb_dim,
        hid_dim=hid_dim,
        n_layers=n_layers,
    )


# TODO: Training and testing loops

In [10]:
LF_SOS_INDEX = lf_word2idx['<s>']
LF_EOS_INDEX = lf_word2idx['</s>']
LF_PAD_INDEX = lf_word2idx['<PAD>']

In [11]:
print(LF_SOS_INDEX, LF_EOS_INDEX, LF_PAD_INDEX)

0 5 51


In [12]:
def train(model: nn.Module, train_dataloader: DataLoader, num_epochs: int = 5,
          device: str = "cuda") -> nn.Module:
    """
    Trains a given model on the provided DataLoader using RMSProp and NLLLoss.

    Parameters
    ----------
    model : nn.Module
        The Seq2Seq model to train.
    train_dataloader : DataLoader
        DataLoader containing the training dataset.
    num_epochs : int
        Number of epochs to train the model.
    device : str
        Device to run the training on ("cuda" or "cpu").

    Returns
    ----------
    nn.Module
        The trained model.
    """
    # Define the loss function
    criterion = model.get_loss()
    optimizer = model.get_optimizer()

    # Add a scheduler for dynamic learning rate adjustment
    scheduler = StepLR(optimizer, step_size=5, gamma=0.5)  # Halve LR every 2 epochs
    # Move the model to the appropriate device
    model = model.to(device)
    model.train()  # Set the model to training mode

    for epoch in range(0, num_epochs):
        epoch_loss = 0
        start_time = timer()

        for queries, logical_forms in train_dataloader:
            # Move data to device
            queries = queries.to(device)
            logical_forms = logical_forms.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(queries, logical_forms, 1.0)

            # Compute log probabilities for NLLLoss
            log_probs = outputs.log_softmax(dim=-1)

            # Reshape outputs and targets for loss computation
            log_probs = log_probs.view(-1, log_probs.size(-1))  # [seq_len * batch_size, output_dim]
            targets = logical_forms.view(-1)                   # [seq_len * batch_size]

            # Compute loss
            loss = criterion(log_probs, targets)

            # Backward pass and optimization step
            loss.backward()
            optimizer.step()

            # Accumulate loss for the epoch
            epoch_loss += loss.item()
        end_time = timer()

        # Print epoch loss
        train_loss = epoch_loss / len(train_dataloader)
        print((f"Epoch: {epoch+1}, Train loss: {train_loss:.3f}, "f"Epoch time = {(end_time - start_time):.3f}s"))

    print("Training completed!")
    return model


In [13]:


def evaluate(model: nn.Module, dataloader: DataLoader, device: str = "cuda") -> tuple[int, int]:
    """
    Evaluates the model on the provided DataLoader using greedy decoding.

    Parameters
    ----------
    model : nn.Module
        The trained Seq2Seq model.
    dataloader : DataLoader
        DataLoader containing evaluation data.
    device : str
        Device to run the evaluation on ("cuda" or "cpu").

    Returns
    ----------
    tuple
        Per-token accuracy and exact-match accuracy.
    """
    # Initialize metrics as a dictionary
    metrics = {
        "per_token_count": 0,
        "per_token_total": 0,
        "exact_match_count": 0,
        "total_sequences": 0,
    }
    teacher_forcing = 0.0  # Disabled for evaluation

    model.eval()
    with torch.no_grad():
        for input, query in dataloader:
            preds = []
            truth = []
            exact_match_flag = True  # For exact-match accuracy

            # Move data to the device
            input, query = input.to(device), query.to(device)

            # Forward pass with teacher forcing disabled
            logits = model(input, query, teacher_forcing_ratio=teacher_forcing)
            pred = torch.argmax(logits, dim=-1)  # Get predictions
            flattened_pred = pred.view(-1)
            flattened_query = query.view(-1)
            # for p, q in zip(pred[1:].squeeze(), query[1:].squeeze()):
            for p, q in zip(flattened_pred, flattened_query):
                if q.item() != LF_EOS_INDEX and p != q:
                    exact_match_flag = False  # Mark sequence as incorrect if one token mismatches

                if q.item() == LF_EOS_INDEX:  # Stop when <EOS> is encountered
                    break

                if p == q:  # Token matches
                    metrics["per_token_count"] += 1

                metrics["per_token_total"] += 1

                preds.append(query_idx2word[p.item()])
                truth.append(query_idx2word[q.item()])

            # Update exact match count if the sequence matches entirely
            if exact_match_flag:
                metrics["exact_match_count"] += 1

            # Increment total sequences
            metrics["total_sequences"] += 1

    # Compute accuracies
    per_token_acc = metrics["per_token_count"] / metrics["per_token_total"] if metrics["per_token_total"] > 0 else 0.0
    exact_match_acc = metrics["exact_match_count"] / metrics["total_sequences"] if metrics["total_sequences"] > 0 else 0.0

    print(f"Per-token Accuracy: {per_token_acc:.4f}")
    print(f"Exact-match Accuracy: {exact_match_acc:.4f}")

    return per_token_acc, exact_match_acc


# Run this!

In [ ]:
def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Define specific seeds to use
    # seeds = [52]
    seeds = [52,62, 72]
    per_token_accuracies = []
    exact_match_accuracies = []

    for seed in seeds:
        print(f"Running for seed: {seed}")
        # Set seed for reproducibility
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

        # Initialize datasets and dataloaders
        jobs_train, jobs_test = build_datasets()
        dataloader_train, dataloader_test = build_dataloaders(jobs_train, jobs_test, train_batch_size=20)

        # Create and train the model
        model = create_model()
        model = train(model, dataloader_train, num_epochs=20, device=device)

        # Evaluate the model
        test_per_token_accuracy, test_exact_match_accuracy = evaluate(model, dataloader_test, device=device)

        # Store the results
        per_token_accuracies.append(test_per_token_accuracy)
        exact_match_accuracies.append(test_exact_match_accuracy)

        print(f"Seed {seed}: Test Per-token Accuracy = {test_per_token_accuracy:.4f}, "
              f"Test Exact-match Accuracy = {test_exact_match_accuracy:.4f}")

    # Compute averages
    avg_per_token_accuracy = sum(per_token_accuracies) / len(seeds)
    avg_exact_match_accuracy = sum(exact_match_accuracies) / len(seeds)

    # Print average results
    print(f"Average Per-token Accuracy over seeds {seeds}: {avg_per_token_accuracy:.4f}")
    print(f"Average Exact-match Accuracy over seeds {seeds}: {avg_exact_match_accuracy:.4f}")


# Run the main function
main()


Running for seed: 52
Epoch: 1, Train loss: 2.007, Epoch time = 7.353s
Epoch: 2, Train loss: 0.916, Epoch time = 3.473s
Epoch: 3, Train loss: 0.803, Epoch time = 3.719s
Epoch: 4, Train loss: 0.697, Epoch time = 3.772s
Epoch: 5, Train loss: 0.615, Epoch time = 4.078s
Epoch: 6, Train loss: 0.550, Epoch time = 3.356s
Epoch: 7, Train loss: 0.508, Epoch time = 3.349s
Epoch: 8, Train loss: 0.467, Epoch time = 5.174s
Epoch: 9, Train loss: 0.427, Epoch time = 3.396s
Epoch: 10, Train loss: 0.398, Epoch time = 3.355s
Epoch: 11, Train loss: 0.377, Epoch time = 3.769s
Epoch: 12, Train loss: 0.350, Epoch time = 4.174s
Epoch: 13, Train loss: 0.320, Epoch time = 3.401s
Epoch: 14, Train loss: 0.315, Epoch time = 3.372s
Epoch: 15, Train loss: 0.283, Epoch time = 4.415s
Epoch: 16, Train loss: 0.269, Epoch time = 3.378s
Epoch: 17, Train loss: 0.266, Epoch time = 3.334s
Epoch: 18, Train loss: 0.282, Epoch time = 3.528s
Epoch: 19, Train loss: 0.247, Epoch time = 4.231s


In [ ]:
# def main():
#     device = "cuda" if torch.cuda.is_available() else "cpu"
#     jobs_train, jobs_test = build_datasets()
#     dataloader_train, dataloader_test = build_dataloaders(jobs_train, jobs_test, train_batch_size=20)
#     model = create_model()
#     model = train(model, dataloader_train, num_epochs=5, device=device)
#     test_per_token_accuracy, test_exact_match_accuracy = evaluate(model, dataloader_test, device=device)
#     print(f'Test Per-token Accuracy: {test_per_token_accuracy}')
#     print(f'Test Exact-match Accuracy: {test_exact_match_accuracy}')
# main()